# Plot fractions of missing/badrecons photons

It reads summary files from simulations of pileup and plots   

- fraction of missing photons   
- fraction of badrecons photons   
- flagging success   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import hist
import glob

## Read CSV files for all fluxes and store full information

In [ ]:
# read all csv files in current dir
files = glob.glob('info*.csv')
nfiles = len(files)
# get list of fluxes from the files
fluxes = []
for file in files:
    df = pd.read_csv(file)
    fluxes.append(df['flux[mcrab]'].values[0])
# sort the files by flux
files = [x for _, x in sorted(zip(fluxes, files))]

# inititalize numpy array of fluxes
fluxes = np.zeros(nfiles)
# initialize numpy array of mean and std
means = np.zeros(nfiles)
stds = np.zeros(nfiles)

# start figure with axes
fig, ax = plt.subplots(1,1, figsize=(10,6))

# loop over files
for i in range(nfiles):
    file = files[i]
    # read data from csv file
    df = pd.read_csv(file)
    # Read columns "simulation" and "fraction[%lost]" for the given exposure time
    sim = df["simulation"]
    frac = df["fraction_lost[%]"]
    # get data from the table
    exposure = df['exposure[s]'].values[0]
    filter = df['filter'].values[0]
    focus = df['focus'].values[0]
    Nimpacts = int(np.mean(df['Nimpacts'].values))
    Nmissing = int(np.mean(df['Missing'].values))
    flux_mcrab = df['flux[mcrab]'].values[0]
    
    mean = np.mean(frac)
    std = np.std(frac)

    # add flux to array
    fluxes[i] = flux_mcrab
    # add mean and std to array
    means[i] = mean
    stds[i] = std
    
    
# plot fraction lost vs flux in log scale
ax.errorbar(fluxes, means, yerr=stds, fmt='o', color='C0', mfc='white', markersize=5, capsize=5, label='missing photons')
# plot fraction of lost+badrecons vs flux in log scale (multiply by 2: for each badrecon, there is a lost) 
ax.errorbar(fluxes, 2*means, yerr=2*stds, fmt='o', color='black', mfc='white', markersize=5, capsize=5, label='lost+badrecons')

# set vertical line at >0.5 mCrab
ax.axvline(x=0.55, color='red', linestyle='--', label='infoc/defoc', alpha=0.5)

ax.set_xscale('log')
ax.set_yscale('linear')
# do not use power of ten notation
ax.xaxis.get_offset_text().set_visible(False)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0f}'.format(x)))

# set infoc/defoc text
ax.text(0.25, 0.4, 'infoc', fontsize=15, color='black')
ax.text(0.7, 0.4, 'defoc', fontsize=15, color='black')
# draw an arrow from the line to the right
ax.annotate('', xy=(0.65, 0.35), xytext=(1.1, 0.35),
            arrowprops=dict(arrowstyle='<-', lw=1.5, color='black'))
ax.annotate('', xy=(0.3, 0.35), xytext=(0.5, 0.35),
            arrowprops=dict(arrowstyle='->', lw=1.5, color='black'))
ax.set_xlabel('Flux [mCrab]', fontsize=18)
ax.set_ylabel('Fraction [%]', fontsize=18)
ax.legend(fontsize=15)

plt.show()

In [ ]:
# save figure
fig.savefig('Figures/fractions/fraction_lost_vs_flux.png', dpi=300, bbox_inches='tight')
# save figure in pdf
fig.savefig('Figures/fractions/fraction_lost_vs_flux.pdf', dpi=300, bbox_inches='tight')